# Extract covered lines details

### Var declarations

In [3]:
import os
from my_secrets import base_path


paths_dict=  {
    "data_file_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "bug_reports_with_stack_traces_details.json"),
    "gzoltar_files_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "gzoltar_files"),
    "output_file_path": os.path.join(base_path,"DeepDiveBugReportsWithLogs", "data", "coverage_data_gzoltar.json"),
    "tests_analysis_results":  os.path.join(base_path,"DeepDiveBugReportsWithLogs",  "data", "rq1_results.json"),
    "tests_covering_stack_trace_entries_folder":  os.path.join(base_path,"DeepDiveBugReportsWithLogs",  "data", "tests_covering_stack_traces_details_per_bug"),
}

## Reading the data

In [2]:
import utils
import importlib
importlib.reload(utils)

tests_analysis_results  = utils.json_file_to_dict(paths_dict["tests_analysis_results"])
coverage_data = {}

bugs = list(tests_analysis_results["bugs_with_stack_traces"]["bugs_without_failing_tests_in_commons_with_defects4j"].keys()) + list(tests_analysis_results["bugs_with_stack_traces"]["bugs_with_failing_tests_in_commons_with_defects4j"].keys())

for bug in bugs:
    project, bug_id = bug.split("_")
    project_gzoltar_folder =   os.path.join(paths_dict["gzoltar_files_path"], project)
    if not os.path.exists(project_gzoltar_folder):
        print(f"Gzoltar folder not fount for the project {project}. Skipping it!")
        continue
    if not project in coverage_data.keys():
        coverage_data[project] = {}
    bug_gzoltar_folder = os.path.join(project_gzoltar_folder, bug_id)
    if not os.path.exists(bug_gzoltar_folder):
        print("Gzoltar folder not fount for the bugId "+ project+ "-" + bug_id)
        print("Skipping!!!!! ")
        continue
    coverage_data[project][bug_id] = {}
    coverage_data[project][bug_id]["statements_covered_per_test"] = utils.read_matrix_file(bug_gzoltar_folder)
    print("Number of tests in bug "+ project+ "-" + bug_id + " - " + str(len(coverage_data[project][bug_id]["statements_covered_per_test"])))
    coverage_data[project][bug_id]["lines_of_code_obj_list"] = utils.read_spectra_file(bug_gzoltar_folder)
    test_names, test_results = utils.read_tests_file(bug_gzoltar_folder)
    coverage_data[project][bug_id]["test_names"] = test_names
    coverage_data[project][bug_id]["test_results"] = test_results
print("Done")

Number of tests in bug Cli-14 - 539
Number of tests in bug Cli-5 - 103
Number of tests in bug Closure-106 - 2554
Number of tests in bug Closure-125 - 8156
Number of tests in bug Closure-143 - 4616
Number of tests in bug Closure-152 - 6186
Number of tests in bug Closure-2 - 7830
Number of tests in bug Closure-34 - 7413
Number of tests in bug Closure-37 - 7373
Number of tests in bug Closure-55 - 7155
Number of tests in bug Codec-8 - 294
Number of tests in bug Collections-28 - 5280
Number of tests in bug Compress-1 - 69
Number of tests in bug Compress-12 - 315
Number of tests in bug Compress-14 - 317
Number of tests in bug Compress-17 - 350
Number of tests in bug Compress-18 - 351
Number of tests in bug Compress-23 - 466
Number of tests in bug Compress-24 - 935
Number of tests in bug Compress-27 - 1012
Number of tests in bug Compress-31 - 1106
Number of tests in bug Compress-32 - 613
Number of tests in bug Compress-34 - 1228
Number of tests in bug Csv-12 - 206
Number of tests in bug Csv-4

# Getting the general coverage information - covered lines per file and method - stored in data/coverage_data_gzoltar.json
This information is later used to calculate the coverage percentage of the files and methods being analysed

In [13]:
covered_lines_gzoltar = {}
for project in coverage_data.keys():
    print(project)
    for bug_id in coverage_data[project].keys():
        coverage_info = {}
        for index_t, test_coverage in enumerate(coverage_data[project][bug_id]["statements_covered_per_test"]):
            for index_s, statement_instance in enumerate(coverage_data[project][bug_id]["statements_covered_per_test"][index_t]):
                if str(statement_instance) == "1": # 1= covered, 0=not covered
                    lines_of_code_obj_list = coverage_data[project][bug_id]["lines_of_code_obj_list"][index_s]
                    file_name = lines_of_code_obj_list["class_name"].replace(".", "/").replace("$", "/") + ".java"
                    method_name = lines_of_code_obj_list["method_name"] + " - " + lines_of_code_obj_list["method_parameters"]
                    line_number = lines_of_code_obj_list["line_number"]
                    if file_name not in coverage_info.keys():
                        coverage_info[file_name] = {}
                    if method_name not in coverage_info[file_name].keys():
                        coverage_info[file_name][method_name] = []
                    if line_number not in coverage_info[file_name][method_name]:
                        coverage_info[file_name][method_name].append(line_number)
        if project not in covered_lines_gzoltar.keys():
            covered_lines_gzoltar[project] = {}
        covered_lines_gzoltar[project][bug_id] = coverage_info
utils.dict_to_json_file(paths_dict["output_file_path"],covered_lines_gzoltar)
print("Collected info added to the file "+ paths_dict["output_file_path"])

Cli
Closure
Codec
Collections
Compress
Csv
Gson
JacksonCore
JacksonDatabind
Jsoup
JxPath
Lang
Math
Mockito
Time


KeyError: 'gzoltar_file_path'

# Getting the names of the tests that cover each stack trace entry
This information is later used in Ochiai computations

In [7]:
import utils
import importlib
importlib.reload(utils)

bugs_data = utils.json_file_to_dict(paths_dict["data_file_path"])

# TODO: Remove later
tests_analysis_results  = utils.json_file_to_dict(paths_dict["tests_analysis_results"])
bugs = list(tests_analysis_results["bugs_with_stack_traces"]["bugs_without_failing_tests_in_commons_with_defects4j"].keys()) + list(tests_analysis_results["bugs_with_stack_traces"]["bugs_with_failing_tests_in_commons_with_defects4j"].keys())


for bug in bugs:
    print(bug)
    project, bug_id = bug.split("_")
    bug_coverage_data = coverage_data[project][bug_id]
    bug_data = bugs_data[project][bug_id]
    new_file_path = os.path.join(paths_dict["tests_covering_stack_trace_entries_folder"], project, bug_id + ".json") #TODO: Continue
    info = {}
    for st_file in bug_data["stackTraceMethodsDetails"].keys():
        for st_method in bug_data["stackTraceMethodsDetails"][st_file].keys():
            tests_covering_the_method = {}
            for index_t, test_coverage in enumerate(bug_coverage_data["statements_covered_per_test"]):
                test_name = bug_coverage_data["test_names"][index_t]
                for index_s, statement_instance in enumerate(bug_coverage_data["statements_covered_per_test"][index_t]):
                    if str(statement_instance) == "1": # 1= covered, 0=not covered
                        lines_of_code_obj_list = bug_coverage_data["lines_of_code_obj_list"][index_s]
                        file_name = lines_of_code_obj_list["class_name"].replace(".", "/").replace("$", "/") + ".java"
                        method_name = lines_of_code_obj_list["method_name"]
                        line_number = lines_of_code_obj_list["line_number"]
                        if st_file.endswith(file_name) and st_method == method_name:
                            st_method_start_line = bug_data["stackTraceMethodsDetails"][st_file][st_method]["startLine"]
                            st_method_end_line = bug_data["stackTraceMethodsDetails"][st_file][st_method]["endLine"]
                            if st_method_start_line <= line_number <= st_method_end_line:
                                if test_name not in tests_covering_the_method.keys():
                                    tests_covering_the_method[test_name] = [line_number]
                                else:
                                    tests_covering_the_method[test_name].append(line_number)
            if st_file not in info.keys():
                info[st_file] = {}
            info[st_file][st_method] = {}
            info[st_file][st_method]["tests_covering_the_method"] = tests_covering_the_method
    utils.dict_to_json_file(new_file_path, info)
    print(f"Updated info for bug {bug} added to the file")
print("All collected info added to the files in "+ paths_dict["tests_covering_stack_trace_entries_folder"])

Cli_14
Updated info for bug Cli_14 added to the file
Cli_5
Updated info for bug Cli_5 added to the file
Closure_106
Updated info for bug Closure_106 added to the file
Closure_125
Updated info for bug Closure_125 added to the file
Closure_143
Updated info for bug Closure_143 added to the file
Closure_152
Updated info for bug Closure_152 added to the file
Closure_2
Updated info for bug Closure_2 added to the file
Closure_34
Updated info for bug Closure_34 added to the file
Closure_37
Updated info for bug Closure_37 added to the file
Closure_55
Updated info for bug Closure_55 added to the file
Codec_8
Updated info for bug Codec_8 added to the file
Collections_28
Updated info for bug Collections_28 added to the file
Compress_1
Updated info for bug Compress_1 added to the file
Compress_12
Updated info for bug Compress_12 added to the file
Compress_14
Updated info for bug Compress_14 added to the file
Compress_17
Updated info for bug Compress_17 added to the file
Compress_18
Updated info for